In [3]:
# Install newer version of Pytorch
!pip3 install https://download.pytorch.org/whl/cu100/torch-1.1.0-cp36-cp36m-linux_x86_64.whl
!pip3 install https://download.pytorch.org/whl/cu100/torchvision-0.3.0-cp36-cp36m-linux_x86_64.whl

# Cloning ONLY colab-integration branch
from getpass import getpass
import os

user = getpass('Github user')
password = getpass('Githyb password')
os.environ['GITHUB_AUTH'] = user + ':' + password
!git clone -b colab-integration --single-branch "https://$GITHUB_AUTH@github.com/baltekgajda/LegoSortingRecognition.git"

# Navigate to project folder
os.chdir("./LegoSortingRecognition")

# Create dir for models
!cd LegoSortingRecognition
!mkdir models

Github user··········
Githyb password··········
fatal: destination path 'LegoSortingRecognition' already exists and is not an empty directory.
/bin/bash: line 0: cd: LegoSortingRecognition: No such file or directory
mkdir: cannot create directory ‘models’: File exists


In [8]:
# Import necessary libraries
from __future__ import print_function
from __future__ import division
from google.colab import drive
from torchsummary import summary
import torch
import torch.nn as nn
import torch.optim as optim
import utils

from data_loader import load_data
from feature_extraction import initialize_model, train_model
import VGGFactory

In [4]:
# Mount Google Drive to access data
drive.mount('/content/drive')

Go to this URL in a browser: https://accounts.google.com/o/oauth2/auth?client_id=947318989803-6bn6qk8qdgf4n4g3pfee6491hc0brc4i.apps.googleusercontent.com&redirect_uri=urn%3Aietf%3Awg%3Aoauth%3A2.0%3Aoob&scope=email%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdocs.test%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fdrive.photos.readonly%20https%3A%2F%2Fwww.googleapis.com%2Fauth%2Fpeopleapi.readonly&response_type=code

Enter your authorization code:
··········
Mounted at /content/drive


In [4]:
def get_params_to_update(model):
  params_to_update = model.parameters()
  params_to_update = []
  for name, param in model.named_parameters():
    if param.requires_grad:
        params_to_update.append(param)
        
  return params_to_update

In [5]:
def train_and_evaluate_classifier_only(dataloaders_dicts, device, num_of_classes=20, num_of_epochs=10):
  model = VGGFactory.create_model(1, num_of_classes)
  optimizer = optim.SGD(get_params_to_update(model), lr=0.001, momentum=0.9)
  criterion = nn.CrossEntropyLoss()
  model_ft, hist = train_model(model, dataloaders_dict, criterion, optimizer, device, num_epochs=num_of_epochs)
  utils.save_model(model_ft, "vgg-classifier-only")
  return (model_ft, hist)

In [7]:
def train_and_evaluate_last_conv(dataloaders_dicts, device, num_of_classes=20, num_of_epochs=10):
  model = VGGFactory.create_model(1, num_of_classes)
  model.features[18].requires_grad = True
  
  optimizer = optim.SGD(get_params_to_update(model), lr=0.001, momentum=0.9)
  criterion = nn.CrossEntropyLoss()
  model_ft, hist = train_model(model, dataloaders_dict, criterion, optimizer, device, num_epochs=num_of_epochs)
  utils.save_model(model_ft, "vgg-last_conv")
  return (model_ft, hist)

In [23]:
NUM_OF_CLASSES = 20
NUM_OF_EPOCHS = 1
INPUT_SIZE = 224
DATA_DIR = '/content/drive/My Drive/Studia/SNR/lego/'

print("Initializing Datasets and Dataloaders...")
dataloaders_dict = load_data(DATA_DIR, INPUT_SIZE, batch_size=32, num_workers=4)


if (torch.cuda.is_available() == False):
  raise RuntimeError("GPU is not available!")   
device = torch.device("cuda:0")

torch.cuda.current_device()
# train_and_evaluate_classifier_only(dataloaders_dict, device, num_of_classes=NUM_OF_CLASSES, num_of_epochs=NUM_OF_EPOCHS)
train_and_evaluate_last_conv(dataloaders_dict, device, num_of_classes=NUM_OF_CLASSES, num_of_epochs=NUM_OF_EPOCHS)

Initializing Datasets and Dataloaders...
Epoch 0/0
----------
train Loss: 2.8969 Acc: 0.1283
Param:  weight , values:  Parameter containing:
tensor([[ 8.9767e-03,  7.6323e-03,  9.1640e-03,  ..., -1.2716e-03,
         -1.3617e-02, -9.9163e-03],
        [-1.4582e-02,  1.4280e-04, -1.1488e-02,  ..., -8.0286e-04,
          9.7327e-03,  1.2744e-02],
        [-3.8492e-03,  7.2404e-05,  3.4531e-03,  ...,  1.4314e-02,
          7.2706e-03,  9.3963e-03],
        ...,
        [ 9.8318e-03,  6.7536e-03, -3.9300e-03,  ...,  1.2691e-03,
          7.3409e-03,  6.1167e-03],
        [-1.4612e-02,  9.0570e-03, -9.5957e-03,  ...,  1.5217e-02,
         -1.8838e-03, -6.1411e-03],
        [ 4.8502e-04,  1.5325e-02,  7.3305e-03,  ..., -8.9848e-03,
          6.4386e-04, -5.1271e-03]], device='cuda:0', requires_grad=True)
Param:  bias , values:  Parameter containing:
tensor([ 0.0083, -0.0142,  0.0012, -0.0134, -0.0001,  0.0090,  0.0103, -0.0144,
        -0.0084,  0.0035,  0.0023,  0.0263, -0.0107,  0.0042,  0

(VGG(
   (features): Sequential(
     (0): Conv2d(3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     (1): ReLU(inplace)
     (2): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
     (3): Conv2d(64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     (4): ReLU(inplace)
     (5): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
     (6): Conv2d(128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     (7): ReLU(inplace)
     (8): Conv2d(256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     (9): ReLU(inplace)
     (10): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
     (11): Conv2d(256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     (12): ReLU(inplace)
     (13): Conv2d(512, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
     (14): ReLU(inplace)
     (15): MaxPool2d(kernel_size=2, stride=2, padding=0, dilation=1, ceil_mode=False)
     (16): Co

In [0]:
# num_classes = 20
# num_of_epochs = 50
# input_size = 224

# model = VGGFactory.create_model(1, num_classes)
# for name, param in model.named_parameters():
#     if param.requires_grad:
#         print(name)

# print()

# simple_model = VGGFactory.simplify_model(model)
# for name, param in simple_model.named_parameters():
#     if param.requires_grad:
#         print(name)

# print("Initializing Datasets and Dataloaders...")
# dataloaders_dict = load_data(data_dir, input_size, 0.3, 0.1, 0.1)

# params_to_update = model.parameters()
# print("Params to learn:")
# params_to_update = []
# for name, param in model.named_parameters():
#     if param.requires_grad:
#         params_to_update.append(param)
#         print("\t", name)

# optimizer_ft = optim.SGD(params_to_update, lr=0.001, momentum=0.9)

# # Setup the loss fxn
# criterion = nn.CrossEntropyLoss()

# # Detect if we have a GPU available
# device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")

# # Necessary to run script on Windows
# torch.cuda.current_device()

# # Train and evaluate
# model_ft, hist = train_model(model, dataloaders_dict, criterion, optimizer_ft, device, num_epochs=num_of_epochs)

# # Save trained model
# utils.save_model(model_ft, "./models")
